# Simple JSON-LD Framing

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import json, io
import s3fs
import kglab
from rdflib import Graph  #, plugin
from pyld import jsonld

In [2]:
def citation_frame(fn, frame):
    with open(fn, 'rb') as f:
        #return json.loads(f.read().replace('\n',' '))
        try:
            jld = json.loads(f.read().decode("utf-8", "ignore").replace('\n',' '))
            framed = jsonld.frame(jld, frame)
            return framed   # ['citation']
        except:
            return ""

In [3]:
fr = {"@context":{"@vocab": "https://schema.org/"},
      "@type": {},
      "@id": {},
      "@explicit": "true",
      "@requireAll": "true",
      "name": {},
      "identifier": {},
      "description": {}
      };

In [4]:
f = citation_frame("./dataGraphs/001413a5f2127fc1385d8f59e5a3d11ad9cfe630.jsonld", fr)
json_formatted_str = json.dumps(f, indent=2)
print(json_formatted_str)

{
  "@context": {
    "@vocab": "https://schema.org/"
  },
  "@id": "https://edmo.seadatanet.org/report/5102",
  "@type": "Organization",
  "description": "Oxford Archaeology (OA) has a 40-year history of quality, innovation and service. As a multi-disciplinary practice, OA covers all aspects of the historic environment, providing everything from archaeological excavations and survey, landscape and buildings studies, complex geospatial data management systems to heritage consultancy services. Most archaeological work today is carried out in advance of development. Oxford Archaeology is one of the leading service-providers in this field, with thousands of clients who recognise us as experts in excavation and heritage management. As a registered charity with a heritage and education focus, we also care about helping people to discover and enjoy their heritage. We have welcomed many thousands of visitors to our excavation sites on open days and tours, and volunteers have taken part in man

### Now do some jq magic

*  find all keys at any level: 'paths | select(.[-1] == "type")'
*  jq.compile('".@context" += ["csdco", "https://opencoredata.org/voc/csdco/v1/"]')

In [5]:
import jq

# convert string back to json object
co = json.loads(json_formatted_str)

# program1: Just a test..   REMOVE 
# op1 = jq.compile('.description += " THIS IS A TEST"').input(co).all()

# op1 = jq.compile('.["@context"] += {"csdco": "https://opencoredata.org/voc/csdco/v1/"}').input(co).all()
op1 = jq.compile('del(.["@context"]) ').input(co).all()
op2 = jq.compile('map(.["id"] = .["@id"] | del(.["@id"]) )').input(op1).all()


print(json.dumps(op2[0][0], indent=2))


{
  "@type": "Organization",
  "description": "Oxford Archaeology (OA) has a 40-year history of quality, innovation and service. As a multi-disciplinary practice, OA covers all aspects of the historic environment, providing everything from archaeological excavations and survey, landscape and buildings studies, complex geospatial data management systems to heritage consultancy services. Most archaeological work today is carried out in advance of development. Oxford Archaeology is one of the leading service-providers in this field, with thousands of clients who recognise us as experts in excavation and heritage management. As a registered charity with a heritage and education focus, we also care about helping people to discover and enjoy their heritage. We have welcomed many thousands of visitors to our excavation sites on open days and tours, and volunteers have taken part in many of our projects. Our vision is to be at the forefront of advancing knowledge about the past and working in 